In [1]:
!oci --version

3.63.3


In [25]:
user_input = "qué es llama?"

In [26]:
import oci
import json
from oci.auth.signers import get_resource_principals_signer

# === Config ===
REGION = "us-chicago-1"
SERVICE_ENDPOINT = f"https://inference.generativeai.{REGION}.oci.oraclecloud.com"
COMPARTMENT_ID = "ocid1.compartment.oc1..aaaaaaaawy5gpq67r3rxc7yf6yn7oesvvgr56dqg7o5y3f5ndzt2mddtj46a"
MODEL_ID = "ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceya6dvgvvj3ovy4lerdl6fvx525x3yweacnrgn4ryfwwcoq"

# === Signer===
signer = get_resource_principals_signer()
cfg = {"region": REGION}

# (opcional)
if MODEL_ID is None:
    from oci.generative_ai import GenerativeAiClient
    genai = GenerativeAiClient(config=cfg, signer=signer)
    models = genai.list_models(
        compartment_id=COMPARTMENT_ID,
        capability=["CHAT"],
        lifecycle_state="ACTIVE"
    ).data.items
    assert models, "No hay modelos CHAT visibles en el compartimento. Revisa permisos/compartimento."
    MODEL_ID = models[0].id
    print("Usando modelo:", MODEL_ID)

# === Cliente de inferencia ===
inf = oci.generative_ai_inference.GenerativeAiInferenceClient(
    config=cfg, signer=signer, service_endpoint=SERVICE_ENDPOINT
)

# === Prompt del usuario ===
user_input = user_input

# --- Construcción del request ---
content = oci.generative_ai_inference.models.TextContent(text=user_input)
message = oci.generative_ai_inference.models.Message(role="USER", content=[content])

chat_request = oci.generative_ai_inference.models.GenericChatRequest(
    api_format=oci.generative_ai_inference.models.BaseChatRequest.API_FORMAT_GENERIC,
    messages=[message],
    max_tokens=600,
    temperature=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    top_p=0.75,
)

chat_detail = oci.generative_ai_inference.models.ChatDetails(
    serving_mode=oci.generative_ai_inference.models.OnDemandServingMode(model_id=MODEL_ID),
    chat_request=chat_request,
    compartment_id=COMPARTMENT_ID,
)

# === Llamada ===
resp = inf.chat(chat_detail)

# === Resultado ===
choices = resp.data.chat_response.choices
response_text = choices[0].message.content[0].text if choices else "No se generó respuesta."
print(json.dumps({"response": response_text}, indent=2, ensure_ascii=False))

{
  "response": "\"Llama\" puede tener diferentes significados dependiendo del contexto. Aquí te explico los más comunes:\n\n1. **Animal**: La llama es un mamífero camélido originario de Sudamérica, especialmente de los Andes (Perú, Bolivia, Chile, Argentina). Es un animal doméstico utilizado principalmente como animal de carga y por su lana. Es pariente de la alpaca, el guanaco y la vicuña. Las llamas son conocidas por su temperamento tranquilo, aunque pueden escupir si se sienten amenazadas o molestas.\n\n2. **Fuego o flama**: En español, \"llama\" también significa \"flama\" o la parte visible del fuego. Por ejemplo: \"La llama de la vela es muy brillante\".\n\n3. **Llamada telefónica (en algunos países)**: En ciertos países de habla hispana, como México, \"llama\" puede ser una forma coloquial de referirse a una llamada telefónica. Por ejemplo: \"¿Me haces una llama cuando llegues?\"\n\n4. **Llama como verbo**: Es la tercera persona del singular del verbo \"llamar\" en presente. Po